# W Plot example

We demostrate an use of the validphys API directly in Python without going through the reportengine loop.

We produce a scatter plot of the predictions for $W^+$ vs $W^-$ cross sections for each replica in NNPDF 3.0 and 3.1, and compare the result with the experimental error.

In [1]:
from validphys.loader import Loader
from validphys.core import DataSetSpec, PDF, PDFSETS
from validphys.api import API

In [2]:
l = Loader()
thno = 53
dsname = 'ATLASWZTOT13TEV81PB'
cfactors = ['NRM', 'QCD']
pdfs = ['NNPDF30_nnlo_as_0118', 'NNPDF31_nnlo_as_0118']
inp = {
    'pdfs': pdfs,
    'dataset_input': {'dataset': dsname, 'cfac': cfactors},   'theoryid': 53,
    'use_cuts': 'nocuts'
}

In [ ]:
ds = API.dataset(**inp)
resd, res30, res31 = API.one_or_more_results(**inp)

In [ ]:
from validphys.plotoptions import get_info

In [ ]:
info = get_info(ds)

In [ ]:
info.y_label

In [ ]:

xlabel, ylabel, *_ = info.extra_labels[' ']
units = info.y_label

In [ ]:
info.extra_labels

In [ ]:
units

In [ ]:
import numpy as np
import numpy.linalg as la

In [ ]:
cov = resd.covmat[:2,:2]
cov

In [ ]:
vals, vects = la.eigh(cov)

In [ ]:
vals = np.sqrt(vals)

In [ ]:
cov

In [ ]:
cv = resd.central_value[:2]

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

In [ ]:
vects

In [ ]:
vals, vects = la.eigh(resd.covmat[:2,:2])

In [ ]:
vals = np.sqrt(vals)

In [ ]:
from validphys.app import app
from validphys import mplstyles

plt.style.use(str(mplstyles.smallstyle))

angle = np.arctan2(vects[0,1], vects[0,0])*180/np.pi



el1s = Ellipse(xy=cv, angle=angle, width=2*vals[0], height=2*vals[1], zorder=1)
el2s = Ellipse(xy=cv, angle=angle, width=4*vals[0], height=4*vals[1], zorder=1)

#fig = plt.figure(0)
#ax = fig.add_subplot(111, aspect='equal')

fig, ax = plt.subplots()

color30 = next(ax._get_lines.prop_cycler)['color']
color31 = next(ax._get_lines.prop_cycler)['color']

ax.plot(*res30._rawdata[:2,:], 'o', label=pdfs[0], color=color30)
ax.plot(*res31._rawdata[:2,:], 'o', label=pdfs[1], color=color31)

ax.axvline(res30.central_value[0], color=color30, linewidth=0.5)
ax.axhline(res30.central_value[1], color=color30,linewidth=0.5)

ax.axvline(res31.central_value[0], color=color31,linewidth=0.5)
ax.axhline(res31.central_value[1], color=color31,linewidth=0.5)

ax.axvline(resd.central_value[0], color='grey',linewidth=0.5)
ax.axhline(resd.central_value[1], color='grey',linewidth=0.5)


ax.add_artist(el1s)
ax.add_artist(el2s)

el1s.set_facecolor('black')
el1s.set_alpha(0.5)
el1s.set(label = "$1 \sigma$")

el2s.set_facecolor('black')
el2s.set_alpha(0.2)

ax.set_xlabel("%s [fb]"%xlabel)
ax.set_ylabel("%s [fb]"%ylabel)

ax.set_axisbelow(True)
ax.set_title("$W^{\pm}$ (NNLO)")
ax.legend()


In [ ]:
fig.savefig("wabs.pdf")